In [1]:
import yfinance as yf

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [ ]:
import requests
import pandas as pd
from io import StringIO

url_tesla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_tesla = requests.get(url_tesla).text

all_tables = pd.read_html(StringIO(html_tesla))

tesla_revenue = None
for table in all_tables:
    if "Tesla Quarterly Revenue" in table.columns[0]:
        tesla_revenue = table
        break

if tesla_revenue is None:
    raise ValueError("Could not find Tesla Quarterly Revenue table on the page.")

tesla_revenue.columns = ["Date", "Revenue"]
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].replace(r"\$|,", "", regex=True)
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]

print(tesla_revenue.head())


In [5]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


In [ ]:
url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_gme = requests.get(url_gme).text
soup = BeautifulSoup(html_gme, "html.parser")

for table in soup.find_all("table"):
    if "GameStop Quarterly Revenue" in table.text:
        gme_revenue = pd.read_html(str(table))[0]
        break

gme_revenue.columns = ["Date","Revenue"]
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r"\$|,", "", regex=True)
gme_revenue.dropna(inplace=True)
gme_revenue.head()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Close'], name="Tesla Stock Price"))
fig.add_trace(go.Bar(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'].astype(float)/1e6,
                     name="Tesla Quarterly Revenue (Millions)"))

fig.update_layout(title="Tesla Stock Price vs Quarterly Revenue",
                  xaxis_title="Date", yaxis_title="Price / Revenue",
                  barmode="overlay")
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gme_data['Date'], y=gme_data['Close'], name="GameStop Stock Price"))
fig.add_trace(go.Bar(x=gme_revenue['Date'], y=gme_revenue['Revenue'].astype(float)/1e6,
                     name="GameStop Quarterly Revenue (Millions)"))

fig.update_layout(title="GameStop Stock Price vs Quarterly Revenue",
                  xaxis_title="Date", yaxis_title="Price / Revenue",
                  barmode="overlay")
fig.show()
